# In this notebook, we explore different ways to impute the missing values

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# import the functions
from impute import *

Load the data 

In [2]:
# load the data
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
# drop the columns that have significant missing entries
train = train.drop(['FireplaceQu','Fence','Alley','MiscFeature','PoolQC'],axis=1)
# get numerical and categorical columns
num_cols = list(train._get_numeric_data().columns)
num_cols.remove('Id')
cat_cols = list(set(train.columns) - set(num_cols))
# # Alternatively, we can use this method (and then remove SalePrice from num_cols and Id from cat_cols)
# cat_cols = list((train.dtypes[train.dtypes == 'object']).index)
# num_cols = list(set(train.columns) - set(cat_cols))
num_cols.remove('SalePrice')
cat_cols.remove('Id')

In [3]:
# split the features and the response
x_train = train.drop(['Id','SalePrice'],axis=1)
y_train = train['SalePrice']
x_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,61,0,0,0,0,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,42,0,0,0,0,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,35,272,0,0,0,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,84,0,0,0,0,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,40,0,0,0,0,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,60,0,0,0,0,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,112,0,0,0,0,4,2010,WD,Normal


Make ordinal encodings for categorical variables

In [4]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
enc.fit(x_train[cat_cols])
ord_cat = pd.DataFrame(enc.transform(x_train[cat_cols]),columns = x_train[cat_cols].columns)
x_train[cat_cols] = ord_cat
x_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,3.0,65.0,8450,1.0,3.0,3.0,0.0,4.0,0.0,...,61,0,0,0,0,0,2,2008,8.0,4.0
1,20,3.0,80.0,9600,1.0,3.0,3.0,0.0,2.0,0.0,...,0,0,0,0,0,0,5,2007,8.0,4.0
2,60,3.0,68.0,11250,1.0,0.0,3.0,0.0,4.0,0.0,...,42,0,0,0,0,0,9,2008,8.0,4.0
3,70,3.0,60.0,9550,1.0,0.0,3.0,0.0,0.0,0.0,...,35,272,0,0,0,0,2,2006,8.0,0.0
4,60,3.0,84.0,14260,1.0,0.0,3.0,0.0,2.0,0.0,...,84,0,0,0,0,0,12,2008,8.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,3.0,62.0,7917,1.0,3.0,3.0,0.0,4.0,0.0,...,40,0,0,0,0,0,8,2007,8.0,4.0
1456,20,3.0,85.0,13175,1.0,3.0,3.0,0.0,4.0,0.0,...,0,0,0,0,0,0,2,2010,8.0,4.0
1457,70,3.0,66.0,9042,1.0,3.0,3.0,0.0,4.0,0.0,...,60,0,0,0,0,2500,5,2010,8.0,4.0
1458,20,3.0,68.0,9717,1.0,3.0,3.0,0.0,4.0,0.0,...,0,112,0,0,0,0,4,2010,8.0,4.0


Which columns have missing values?

In [5]:
pd.DataFrame(x_train.isnull().sum()[x_train.isnull().sum()>0].sort_values(ascending=False),columns=['num_missing'])

,num_missing
LotFrontage,259
GarageType,81
GarageYrBlt,81
GarageFinish,81
GarageQual,81
GarageCond,81
BsmtExposure,38
BsmtFinType2,38
BsmtQual,37
BsmtCond,37


## Imputation Scheme: 

#### First, we naively impute the missing values in the data frame so that we don't have missing values when we train the machine learning model. Then we randomly mask some entries with existing values and train models to predict them. The performance is also evaluated on the masked entries (This may not be a good metric).

## 1. Mean/Mode Imputation (as a baseline)

Normalize the numerical columns before training the model

In [6]:
from sklearn.preprocessing import StandardScaler
normalizer = StandardScaler()
x_train[num_cols] = normalizer.fit_transform(x_train[num_cols])
x_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0.073375,3.0,-0.208034,-0.207142,1.0,3.0,3.0,0.0,4.0,0.0,...,0.216503,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,0.138777,8.0,4.0
1,-0.872563,3.0,0.409895,-0.091886,1.0,3.0,3.0,0.0,2.0,0.0,...,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-0.489110,-0.614439,8.0,4.0
2,0.073375,3.0,-0.084449,0.073480,1.0,0.0,3.0,0.0,4.0,0.0,...,-0.070361,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,0.990891,0.138777,8.0,4.0
3,0.309859,3.0,-0.414011,-0.096897,1.0,0.0,3.0,0.0,0.0,0.0,...,-0.176048,4.092524,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,-1.367655,8.0,0.0
4,0.073375,3.0,0.574676,0.375148,1.0,0.0,3.0,0.0,2.0,0.0,...,0.563760,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,2.100892,0.138777,8.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.073375,3.0,-0.331620,-0.260560,1.0,3.0,3.0,0.0,4.0,0.0,...,-0.100558,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,0.620891,-0.614439,8.0,4.0
1456,-0.872563,3.0,0.615871,0.266407,1.0,3.0,3.0,0.0,4.0,0.0,...,-0.704483,-0.359325,-0.116339,-0.270208,-0.068692,-0.087688,-1.599111,1.645210,8.0,4.0
1457,0.309859,3.0,-0.166839,-0.147810,1.0,3.0,3.0,0.0,4.0,0.0,...,0.201405,-0.359325,-0.116339,-0.270208,-0.068692,4.953112,-0.489110,1.645210,8.0,4.0
1458,-0.872563,3.0,-0.084449,-0.080160,1.0,3.0,3.0,0.0,4.0,0.0,...,-0.704483,1.473789,-0.116339,-0.270208,-0.068692,-0.087688,-0.859110,1.645210,8.0,4.0


Impute mean for each numerical variable and mode for each categorical variable

In [7]:
mm_imputed_df, mean_mse,mode_ham = mean_mode_eval(x_train,num_cols,cat_cols)
np.sum(mm_imputed_df.isna().sum() > 0)

0

## 2. Machine Learning Models

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
ols = LinearRegression()
log = LogisticRegression(random_state = 0,max_iter = 5000)
_,ols_mse,log_ham = ml_all_steps(x_train,num_cols,cat_cols,ols,log)

/opt/anaconda3/envs/clvr_impl/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/opt/anaconda3/envs/clvr_impl/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/

In [9]:
from sklearn.linear_model import ElasticNetCV
from sklearn.neighbors import KNeighborsClassifier
enet = ElasticNetCV(alphas=[1e-3, 1e-2, 1e-1, 1],max_iter = 5000)
knnc = KNeighborsClassifier()
_,enet_mse,knnc_ham = ml_all_steps(x_train,num_cols,cat_cols,enet,knnc)

In [10]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
# from sklearn.neighbors import KNeighborsClassifier
knnr = KNeighborsRegressor()
knnc = KNeighborsClassifier()
_,knnr_mse,knnc_ham = ml_all_steps(x_train,num_cols,cat_cols,knnr,knnc)

In [11]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
rfr = RandomForestRegressor()
rfc = RandomForestClassifier()
_,rfr_mse,rfc_ham = ml_all_steps(x_train,num_cols,cat_cols,rfr,rfc)

## Evaulation

In [12]:
# get the columns with missing entries
m = x_train.isnull().sum()
m = m[m>0]
missing_cols = set(m.index)
# split into numerical and categorical columns
missing_num = missing_cols & set(num_cols)
missing_cat = missing_cols & set(cat_cols)

In [13]:
mse = pd.DataFrame(columns = num_cols,dtype=float)
mse.loc['OLS'] = ols_mse
mse.loc['Elastic Net'] = enet_mse
mse.loc['KNN'] = knnr_mse
mse.loc['Random Forests'] = rfr_mse
mse.loc['Mean Impute'] = mean_mse
rmse = np.sqrt(mse)
rmse.loc['Best Model'] = rmse.idxmin()
# rmse.style.set_caption("Root Mean Square Error")
rmse[missing_num]

,LotFrontage,GarageYrBlt,MasVnrArea
OLS,0.78078,0.387279,0.782557
Elastic Net,0.787229,0.405212,0.790371
KNN,0.81094,0.503734,0.611551
Random Forests,0.313453,0.185981,0.254603
Mean Impute,1.260714,1.032031,1.048522
Best Model,Random Forests,Random Forests,Random Forests


In [14]:
ham = pd.DataFrame(columns = cat_cols,dtype=float)
ham.loc['Logistic'] = log_ham
ham.loc['KNN'] = knnc_ham
ham.loc['Random Forests'] = rfc_ham
ham.loc['Mode Impute'] = mode_ham
ham.loc['Best Model'] = ham.idxmin()
ham[missing_cat]

,BsmtExposure,BsmtFinType2,GarageQual,GarageCond,BsmtCond,GarageType,GarageFinish,MasVnrType,Electrical,BsmtFinType1,BsmtQual
Logistic,0.220657,0.075893,0.044776,0.032864,0.082524,0.13615,0.306306,0.221198,0.068376,0.296296,0.181416
KNN,0.239437,0.147321,0.059701,0.032864,0.087379,0.122066,0.324324,0.281106,0.07265,0.375661,0.19469
Random Forests,0.084507,0.017857,0.044776,0.032864,0.07767,0.084507,0.099099,0.046083,0.055556,0.089947,0.09292
Mode Impute,0.333333,0.151786,0.069652,0.046948,0.092233,0.347418,0.531532,0.428571,0.08547,0.719577,0.553097
Best Model,Random Forests,Random Forests,Logistic,Logistic,Random Forests,Random Forests,Random Forests,Random Forests,Random Forests,Random Forests,Random Forests


## 3. Iteratively Apply Machine Learning Models

#### KNN

In [15]:
_,iter_knn_mse,iter_knn_ham = iter_ml_impute_eval(x_train,num_cols,cat_cols,knnr,knnc,4)

In [16]:
iter_knn_mse[missing_num] ** 0.5

,LotFrontage,GarageYrBlt,MasVnrArea
0,0.990672,0.581854,0.738174
1,0.988538,0.569543,0.730060
2,0.990905,0.575310,0.727581
3,0.994201,0.577454,0.728105


In [17]:
print('One shot')
rmse[missing_num]

One shot


,LotFrontage,GarageYrBlt,MasVnrArea
OLS,0.78078,0.387279,0.782557
Elastic Net,0.787229,0.405212,0.790371
KNN,0.81094,0.503734,0.611551
Random Forests,0.313453,0.185981,0.254603
Mean Impute,1.260714,1.032031,1.048522
Best Model,Random Forests,Random Forests,Random Forests


In [18]:
iter_knn_ham[missing_cat]

,BsmtExposure,BsmtFinType2,GarageQual,GarageCond,BsmtCond,GarageType,GarageFinish,MasVnrType,Electrical,BsmtFinType1,BsmtQual
0,0.253521,0.142857,0.069652,0.042254,0.077670,0.126761,0.216216,0.225806,0.055556,0.259259,0.185841
1,0.253521,0.151786,0.069652,0.042254,0.082524,0.126761,0.243243,0.267281,0.055556,0.306878,0.199115
2,0.253521,0.151786,0.069652,0.042254,0.082524,0.126761,0.238739,0.262673,0.055556,0.306878,0.203540
3,0.253521,0.151786,0.069652,0.042254,0.082524,0.126761,0.238739,0.262673,0.055556,0.312169,0.203540


In [19]:
ham[missing_cat]

,BsmtExposure,BsmtFinType2,GarageQual,GarageCond,BsmtCond,GarageType,GarageFinish,MasVnrType,Electrical,BsmtFinType1,BsmtQual
Logistic,0.220657,0.075893,0.044776,0.032864,0.082524,0.13615,0.306306,0.221198,0.068376,0.296296,0.181416
KNN,0.239437,0.147321,0.059701,0.032864,0.087379,0.122066,0.324324,0.281106,0.07265,0.375661,0.19469
Random Forests,0.084507,0.017857,0.044776,0.032864,0.07767,0.084507,0.099099,0.046083,0.055556,0.089947,0.09292
Mode Impute,0.333333,0.151786,0.069652,0.046948,0.092233,0.347418,0.531532,0.428571,0.08547,0.719577,0.553097
Best Model,Random Forests,Random Forests,Logistic,Logistic,Random Forests,Random Forests,Random Forests,Random Forests,Random Forests,Random Forests,Random Forests


#### Random Forests

In [20]:
_,iter_rfr_mse,iter_rfc_ham = iter_ml_impute_eval(x_train,num_cols,cat_cols,rfr,rfc,4)

In [21]:
iter_rfr_mse[missing_num]

,LotFrontage,GarageYrBlt,MasVnrArea
0,0.243898,0.043339,0.087477
1,0.381994,0.055062,0.174880
2,0.450194,0.080452,0.218567
3,0.493828,0.091365,0.257503


In [22]:
iter_rfc_ham[missing_cat]

,BsmtExposure,BsmtFinType2,GarageQual,GarageCond,BsmtCond,GarageType,GarageFinish,MasVnrType,Electrical,BsmtFinType1,BsmtQual
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
